# Model Save - MLP (AE Min 8 Node)

### Import List

In [1]:
import matplotlib.pyplot as plt
import mglearn
import numpy as np
import os
import random
import pickle

C:\Users\KSY\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\KSY\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Method Definition

In [2]:
base_dir = "..\\..\\..\\data\\BLE_data\\"
def read_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "raw_data\\in_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "raw_data\\out_" + mode + "_" + str(set_num) + '.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def read_denoised_test_data(set_num, model_name, mode):
    if model_name == 5:
        raw_data = np.loadtxt(base_dir + "filtered_data\\in_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    elif model_name == 6:
        raw_data = np.loadtxt(base_dir + "filtered_data\\out_" + mode + "_" + str(set_num) + '_denoised.csv', delimiter=',', dtype=np.float32)
    b_data    = raw_data[:,0:3]
    cell_data = raw_data[:,3:13]

    return b_data, cell_data

def make_plot(y, num, begin, end):  
    fig = plt.figure(figsize=(15, 5))
    fig.patch.set_facecolor('xkcd:white')
    x = range(len(y))
    plt.bar(x, y, color="blue")
    plt.axis([0, num, begin, end])
    plt.show()

## Start

In [3]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-in data
model_name = 5
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_in_train_noisy, y_in_train_noisy = read_test_data(train_set_num, model_name, "train")
x_in_train_denoised, y_in_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_in_test_noisy, y_in_test_noisy = read_test_data(test_set_num, model_name, "test")
x_in_test_denoised, y_in_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [4]:
# model_name: 5 (car_in) / 6 (car_out)
# Car-out data
model_name = 6
set_num = 6000
train_set_num = int(set_num * 0.75)
test_set_num = int(set_num * 0.25)

x_out_train_noisy, y_out_train_noisy = read_test_data(train_set_num, model_name, "train")
x_out_train_denoised, y_out_train_denoised = read_denoised_test_data(train_set_num, model_name, "train")
x_out_test_noisy, y_out_test_noisy = read_test_data(test_set_num, model_name, "test")
x_out_test_denoised, y_out_test_denoised = read_denoised_test_data(test_set_num, model_name, "test")

In [5]:
# Car-in
ae_x_in_train_noisy = x_in_train_noisy.astype('float32') / -100.
ae_x_in_train_denoised = x_in_train_denoised.astype('float32') / -100.
ae_x_in_test_noisy = x_in_test_noisy.astype('float32') / -100.
ae_x_in_test_denoised = x_in_test_denoised.astype('float32') / -100.

In [6]:
# Car-out
ae_x_out_train_noisy = x_out_train_noisy.astype('float32') / -100.
ae_x_out_train_denoised = x_out_train_denoised.astype('float32') / -100.
ae_x_out_test_noisy = x_out_test_noisy.astype('float32') / -100.
ae_x_out_test_denoised = x_out_test_denoised.astype('float32') / -100.

## Autoencoder - Denoising (Car in)

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import load_model

autoencoder_01 = load_model('autoencoder_01.h5')

Using TensorFlow backend.


## Autoencoder - Denoising (Car out)

In [8]:
autoencoder_02 = load_model('autoencoder_02.h5')

## Autoencoder - Converting

## Description
- Autoencoder
    - Car in (noisy)를 Autoencoder에 넣은 결과 Car in (denoised)
    - Car in (denoised) 입력
    - Car out (denoised) 정답

In [9]:
autoencoder_03 = load_model('autoencoder_03.h5')

## Autoencoder - Converting

## Description
- Autoencoder
    - Car in (denoised) 입력
    - Car out (denoised) 정답

In [10]:
autoencoder_04 = load_model('autoencoder_04.h5')

## Test with MLP Model

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import time
import numpy as np
import pickle

def one_hot_convert_normal(data, end_line, output_num):
    list_data = []

    for row in range(0, end_line):
        for column in range(0, output_num):
            if data[row][column] == 1:
                list_data.append(column + 1)
    return list_data

# Description
- mlp_01
    - Car in (noisy)를 AE_03에 넣으면 결과 Car out (denoised = converted)
    - Car out (denoised = converted)를 입력 (Input)
- mlp_02
    - Car in (filtered)를 AE_04에 넣어서 결과 Car out (denoised)
    - Car out (denoised)를 입력 (Input)
- mlp_03
    - Car out (noisy)를 AE_02에 넣으면 Car out (denoised)
    - Car out (denoised)를 입력 (Input)
- mlp_04
    - Car out (filtered)를 입력 (Input)
- Output (정답)
    - Car out (filtered)의 레이블

In [12]:
# Training data
output_num = 10
end_line = train_set_num * output_num

oh_y_in_train_denoised = one_hot_convert_normal(y_in_train_denoised, end_line, output_num)
oh_y_in_train_denoised = np.array(oh_y_in_train_denoised)

oh_y_out_train_denoised = one_hot_convert_normal(y_out_train_denoised, end_line, output_num)
oh_y_out_train_denoised = np.array(oh_y_out_train_denoised)

# Testing data
end_line = test_set_num * output_num

oh_y_in_test_denoised = one_hot_convert_normal(y_in_test_denoised, end_line, output_num)
oh_y_in_test_denoised = np.array(oh_y_in_test_denoised)

oh_y_out_test_denoised = one_hot_convert_normal(y_out_test_denoised, end_line, output_num)
oh_y_out_test_denoised = np.array(oh_y_out_test_denoised)

## MLP_01

In [13]:
after_autoencoder_x_train = autoencoder_03.predict(ae_x_in_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_in_test_noisy) * (-100)

# MLP training
mlp_01 = MLPClassifier(random_state=12, hidden_layer_sizes=[50, 100, 50], max_iter=100, alpha=0.001, solver='sgd', verbose=10, tol=0.000000001)
mlp_01.fit(after_autoencoder_x_train, oh_y_out_train_denoised)

# Save model
pickle.dump(mlp_01, open('mlp_01.sav', 'wb'))

# MLP accuracy
print("[MLP] Training data accuracy: {:.3f}".format(mlp_01.score(after_autoencoder_x_train, oh_y_out_train_denoised)))
print("[MLP] Testing data accuracy: {:.3f}".format(mlp_01.score(after_autoencoder_x_test, oh_y_out_test_denoised)))

Iteration 1, loss = 2.76330813
Iteration 2, loss = 2.29766268
Iteration 3, loss = 2.28918918
Iteration 4, loss = 2.27855328
Iteration 5, loss = 2.26249233
Iteration 6, loss = 2.25240197
Iteration 7, loss = 2.24721622
Iteration 8, loss = 2.24198015
Iteration 9, loss = 2.22539192
Iteration 10, loss = 2.22396616
Iteration 11, loss = 2.21353794
Iteration 12, loss = 2.20112230
Iteration 13, loss = 2.19043134
Iteration 14, loss = 2.18029261
Iteration 15, loss = 2.17433812
Iteration 16, loss = 2.16908681
Iteration 17, loss = 2.16054429
Iteration 18, loss = 2.15954761
Iteration 19, loss = 2.13778196
Iteration 20, loss = 2.13703001
Iteration 21, loss = 2.12956877
Iteration 22, loss = 2.12476538
Iteration 23, loss = 2.14892010
Iteration 24, loss = 2.15277505
Iteration 25, loss = 2.13926346
Iteration 26, loss = 2.13612978
Iteration 27, loss = 2.12813471
Iteration 28, loss = 2.11620946
Iteration 29, loss = 2.11100329
Iteration 30, loss = 2.10247286
Iteration 31, loss = 2.10810054
Iteration 32, los

## MLP_02

In [14]:
after_autoencoder_x_train = autoencoder_04.predict(ae_x_in_train_denoised) * (-100)
after_autoencoder_x_test = autoencoder_04.predict(ae_x_in_test_denoised) * (-100)

# MLP training
mlp_02 = MLPClassifier(random_state=12, hidden_layer_sizes=[50, 100, 50], max_iter=100, alpha=0.001, solver='sgd', verbose=10, tol=0.000000001)
mlp_02.fit(after_autoencoder_x_train, oh_y_out_train_denoised)

# Save model
pickle.dump(mlp_02, open('mlp_02.sav', 'wb'))

# MLP accuracy
print("[MLP] Training data accuracy: {:.3f}".format(mlp_02.score(after_autoencoder_x_train, oh_y_out_train_denoised)))
print("[MLP] Testing data accuracy: {:.3f}".format(mlp_02.score(after_autoencoder_x_test, oh_y_out_test_denoised)))

Iteration 1, loss = 1.87435385
Iteration 2, loss = 0.63260047
Iteration 3, loss = 0.43943685
Iteration 4, loss = 0.34122509
Iteration 5, loss = 0.28686291
Iteration 6, loss = 0.27737577
Iteration 7, loss = 0.20904352
Iteration 8, loss = 0.20215978
Iteration 9, loss = 0.19794834
Iteration 10, loss = 0.19440180
Iteration 11, loss = 0.33417309
Iteration 12, loss = 0.19960321
Iteration 13, loss = 0.18994876
Iteration 14, loss = 0.18890954
Iteration 15, loss = 0.18581483
Iteration 16, loss = 0.18370765
Iteration 17, loss = 0.18245001
Iteration 18, loss = 0.18173389
Iteration 19, loss = 0.18109997
Iteration 20, loss = 0.17842423
Iteration 21, loss = 0.17746067
Iteration 22, loss = 0.17377552
Iteration 23, loss = 0.17208190
Iteration 24, loss = 0.17110021
Iteration 25, loss = 0.17191400
Iteration 26, loss = 0.17145865
Iteration 27, loss = 0.17027129
Iteration 28, loss = 0.16956473
Iteration 29, loss = 0.16836703
Iteration 30, loss = 0.16786849
Iteration 31, loss = 0.16778894
Iteration 32, los

## MLP_03

In [15]:
after_autoencoder_x_train = autoencoder_02.predict(ae_x_out_train_noisy) * (-100)
after_autoencoder_x_test = autoencoder_02.predict(ae_x_out_test_noisy) * (-100)

# MLP training
mlp_03 = MLPClassifier(random_state=12, hidden_layer_sizes=[50, 100, 50], max_iter=100, alpha=0.001, solver='sgd', verbose=10, tol=0.000000001)
mlp_03.fit(after_autoencoder_x_train, oh_y_out_train_denoised)

# Save model
pickle.dump(mlp_03, open('mlp_03.sav', 'wb'))

# MLP accuracy
print("[MLP] Training data accuracy: {:.3f}".format(mlp_03.score(after_autoencoder_x_train, oh_y_out_train_denoised)))
print("[MLP] Testing data accuracy: {:.3f}".format(mlp_03.score(after_autoencoder_x_test, oh_y_out_test_denoised)))

Iteration 1, loss = 1.93813670
Iteration 2, loss = 0.53936913
Iteration 3, loss = 0.32182290
Iteration 4, loss = 0.10753402
Iteration 5, loss = 0.08752455
Iteration 6, loss = 0.25634815
Iteration 7, loss = 0.08694983
Iteration 8, loss = 0.08095346
Iteration 9, loss = 0.07805123
Iteration 10, loss = 0.07686258
Iteration 11, loss = 0.20682682
Iteration 12, loss = 0.09310731
Iteration 13, loss = 0.08127813
Iteration 14, loss = 0.07871038
Iteration 15, loss = 0.07696837
Iteration 16, loss = 0.07614683
Iteration 17, loss = 0.07562595
Iteration 18, loss = 0.07586452
Iteration 19, loss = 0.07409629
Iteration 20, loss = 0.07455581
Iteration 21, loss = 0.07458680
Iteration 22, loss = 0.07527694
Iteration 23, loss = 0.07643540
Iteration 24, loss = 0.07421620
Iteration 25, loss = 0.07651230
Iteration 26, loss = 0.07519313
Iteration 27, loss = 0.07424888
Iteration 28, loss = 0.07482788
Iteration 29, loss = 0.15389585
Iteration 30, loss = 0.07593400
Training loss did not improve more than tol=0.000

## MLP_04

In [16]:
# MLP training
mlp_04 = MLPClassifier(random_state=12, hidden_layer_sizes=[50, 100, 50], max_iter=100, alpha=0.001, solver='sgd', verbose=10, tol=0.000000001)
mlp_04.fit(x_out_train_denoised, oh_y_out_train_denoised)

# Save model
pickle.dump(mlp_04, open('mlp_04.sav', 'wb'))

# MLP accuracy
print("[MLP] Training data accuracy: {:.3f}".format(mlp_04.score(x_out_train_denoised, oh_y_out_train_denoised)))
print("[MLP] Testing data accuracy: {:.3f}".format(mlp_04.score(x_out_test_denoised, oh_y_out_test_denoised)))

Iteration 1, loss = 1.99591443
Iteration 2, loss = 0.59806178
Iteration 3, loss = 0.45598832
Iteration 4, loss = 0.26854034
Iteration 5, loss = 0.27752800
Iteration 6, loss = 0.22054149
Iteration 7, loss = 0.15989979
Iteration 8, loss = 0.24964898
Iteration 9, loss = 0.14258549
Iteration 10, loss = 0.13851355
Iteration 11, loss = 0.13745142
Iteration 12, loss = 0.25167155
Iteration 13, loss = 0.12965730
Iteration 14, loss = 0.29887488
Iteration 15, loss = 0.16672318
Iteration 16, loss = 0.11918356
Iteration 17, loss = 0.11886265
Iteration 18, loss = 0.11377988
Iteration 19, loss = 0.11578488
Iteration 20, loss = 0.10460775
Iteration 21, loss = 0.10611254
Iteration 22, loss = 0.11281847
Iteration 23, loss = 0.10528558
Iteration 24, loss = 0.25063383
Iteration 25, loss = 0.13177474
Iteration 26, loss = 0.11506028
Iteration 27, loss = 0.11402268
Iteration 28, loss = 0.30248045
Iteration 29, loss = 0.12870319
Iteration 30, loss = 0.11449467
Iteration 31, loss = 0.10860561
Training loss did